In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import tensorflow as tf


In [2]:

from __future__ import print_function
from scipy.misc import imsave
import numpy as np
import time
from keras.applications import vgg19
from keras import backend as K

from keras.preprocessing import image
from keras.models import Model
from keras.applications.vgg19 import preprocess_input

import matplotlib.pyplot as plt
import math


%matplotlib inline

#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)

#sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
#K.set_session(sess)



Using TensorFlow backend.


In [3]:
content = image.load_img("chien.jpg")
content_array = image.img_to_array(content)


height = content_array.shape[0]
width = content_array.shape[1]


style = image.load_img("style.jpg", target_size=(height,width))
style_array = image.img_to_array(style)

content_array = np.expand_dims(content_array, axis=0)
content_array = preprocess_input(content_array)

style_array = np.expand_dims(style_array, axis=0)
style_array = preprocess_input(style_array)


assert content_array.shape == style_array.shape

generated = K.placeholder(shape=(1,height, width, 3))
content = K.variable(content_array)
style = K.variable(style_array)

print("Height :",height)
print("Width :", width)

Height : 467
Width : 700


In [4]:
model = vgg19.VGG19(weights='imagenet', input_tensor=K.concatenate([content,style,generated], axis=0),include_top=False)

In [5]:
def gram(tensor):
    arranged = K.permute_dimensions(tensor, (2,0,1))
    arranged = K.batch_flatten(arranged)
    return K.dot(arranged, K.transpose(arranged))

In [6]:
def style_loss(gram_style,gram_generated,shape):
    Ml = shape[1] * shape[2]
    Nl = shape[3]
    return K.sum(K.square(gram_generated - gram_style))/(4.*(Ml ** 2) * (Nl ** 2))

In [7]:
def content_loss(content,generated):
    return K.sum(K.square(generated - content))/2.

In [8]:
layers_name = [layer.name for layer in model.layers]
layers_style = ["block1_conv1", "block2_conv1", "block3_conv1", "block4_conv1", "block5_conv1"]
layers_content = ["block4_conv1"]

In [9]:
dict_shape = {}
for name in layers_name:
    dict_shape[name] = model.get_layer(name).output_shape
    
print(dict_shape)

{'block3_pool': (3, 58, 87, 256), 'block4_conv1': (3, 58, 87, 512), 'block2_conv1': (3, 233, 350, 128), 'block3_conv1': (3, 116, 175, 256), 'block2_conv2': (3, 233, 350, 128), 'block4_conv2': (3, 58, 87, 512), 'block2_pool': (3, 116, 175, 128), 'block1_pool': (3, 233, 350, 64), 'block1_conv2': (3, 467, 700, 64), 'block5_conv4': (3, 29, 43, 512), 'block5_pool': (3, 14, 21, 512), 'block4_conv3': (3, 58, 87, 512), 'block3_conv2': (3, 116, 175, 256), 'block5_conv3': (3, 29, 43, 512), 'input_1': (3, 467, 700, 3), 'block5_conv2': (3, 29, 43, 512), 'block1_conv1': (3, 467, 700, 64), 'block4_conv4': (3, 58, 87, 512), 'block3_conv4': (3, 116, 175, 256), 'block5_conv1': (3, 29, 43, 512), 'block4_pool': (3, 29, 43, 512), 'block3_conv3': (3, 116, 175, 256)}


In [10]:
model_dict = dict((name,model.get_layer(name).output) for name in layers_name)

In [11]:
loss_variable = K.variable(0)
content_coeff = 0.0001 #content
style_coeff = 1 #style

loss_style_variable = K.variable(0)
for name in layers_style:
    layer = model_dict[name]
    layer_generated = layer[2,:,:,:]
    layer_style = layer[1,:,:,:]
    
    gram_style = gram(layer_style)
    gram_generated = gram(layer_generated)
    
    shape = dict_shape[name]
    print(shape)
    loss_style_variable +=  (style_loss(gram_style, gram_generated,shape)/len(layers_style))

(3, 467, 700, 64)
(3, 233, 350, 128)
(3, 116, 175, 256)
(3, 58, 87, 512)
(3, 29, 43, 512)


In [12]:
loss_content_variable = K.variable(0.)
for name in layers_content:
    layer = model_dict[name]
    layer_generated = layer[2,:,:,:]
    layer_content = layer[0,:,:,:]
    loss_content_variable += (content_loss(layer_content, layer_generated)) / len(layers_content)
    

In [13]:
total_loss = content_coeff * loss_content_variable + style_coeff * loss_style_variable

In [14]:
gradients = K.gradients(total_loss, generated)[0]



In [15]:
evaluate_loss_grads = K.function([generated], [total_loss, gradients])

In [16]:
def build_img(img_data, height, width):
    img = img_data.reshape((height, width, 3))

    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    
    
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

In [17]:
#Doing Nesterov acceleration

iterate = K.function([generated], [gradients, total_loss])
num_epoch = 1

In [18]:

losses = []

smooth_number = 550.
lambdas = [0]
for i in range(num_epoch + 1):
    last_lambda = lambdas[i]
    new_lambda = (1. + math.sqrt(1. + 4.*(last_lambda**2.) ))/2.
    lambdas.append(new_lambda)
print(lambdas)

gammas = [(1. - lambdas[i])/lambdas[i+1] for i in range(len(lambdas) - 1)]

print(gammas)

[0, 1.0, 1.618033988749895]
[1.0, 0.0]


In [19]:
ys = [0.]
img_data = np.random.uniform(110,150,((1, height, width,3)))
img_data = preprocess_input(img_data)


In [20]:
%%time

for it in range(num_epoch):
    print("Epoch : " , it)
    
    grads_values, loss_values = iterate([img_data])
    print(loss_values)
    
    losses.append(loss_values)
    
    
    ys_1 = img_data - (1. / smooth_number) * grads_values
    ys.append(ys_1)
    current_ys = ys[it]
    
    
    img_data = (1 - gammas[it + 1]) * ys_1 + gammas[it] * ys[it]




Epoch :  0
1.02058e+10
CPU times: user 2min 26s, sys: 4.27 s, total: 2min 30s
Wall time: 40.4 s


In [21]:
def evaluate_loss_grads_from_flatten(flatten_input):   
    correct_input = flatten_input.reshape((1, height, width, 3))
    loss, grads= evaluate_loss_grads([correct_input])
    return loss, grads.flatten().astype('float64')


def getLoss(state):
    def getLoss_input(flatten_input):
        loss_values, grads_values = evaluate_loss_grads_from_flatten(flatten_input)
        state["loss"] = loss_values
        state["gradients"] = grads_values.flatten()
        #print("New state : ",state)
        return loss_values
    return getLoss_input

def getGrads(state):
    def getGrads_input(flatten_input):

        grads = state["gradients"]
        return grads.copy()
    
    return getGrads_input


In [22]:
from scipy.optimize import fmin_l_bfgs_b


state = {}

img_data = np.random.uniform(110,150,((1, height, width,3)))
img_data = preprocess_input(img_data)


loss_lbfgs = []

In [23]:
%%time


for it in range(num_epoch):
    min_input, min_val, info = fmin_l_bfgs_b(getLoss(state), img_data.flatten(),fprime=getGrads(state), maxiter=3, maxfun=5)
    
    loss_lbfgs.append(min_val)
    img_data = min_input.reshape((1, height, width, 3))



CPU times: user 14min 32s, sys: 24 s, total: 14min 56s
Wall time: 4min 11s
